# ACME

# Detailed Plan

Bold letters are the steps covered in ths notebook

1.	Data Ingestion & Splitting. Load Dataset, it is already into train (90%), validation (5%), and test (5%). Then, verify data quality, label integrity and clean.

2.	Descriptive Analytics & KPIs. Run basic stats and define evaluation KPIs like ROUGE-Lsum, summary length and compression ratio to guide model selection and track progress.

3.	Model Selection (Baseline Shootout). Benchmark BERT2BERT vs GPT-2 under the same preprocessing and decoding setup; select the stronger baseline using ROUGE-Lsum plus quick qualitative checks.

4.	**Phase A — Training Optimization. Tune core training hyperparameters (learning rate, weight decay, label smoothing, warmup) with a data subset to establish the best fine-tuned checkpoint.**

5.	**Phase B — Decoding Optimization (Inference Only). Run a full-validation grid over num_beams, length_penalty, no_repeat_ngram_size, max_new_tokens to improve output quality without retraining.**

6.	Targeted Case Reviews. Assess faithfulness and tone on three representative examples (idx 654, 114, 25) to validate real-world usefulness beyond metrics.

7.	Final Diagnostics & Readiness Check. Summarize length distributions, compression ratios, and generated vs. reference comparisons; document risks and guardrails to ensure summaries are concise, accurate, and deployment ready.


**Notebook 2 (BERT2BERT Only)**

This notebook is a simplified variant of the original experimental notebook (Notebook 1).

In Notebook 1, both BERT2BERT and BERT+GPT2 were trained and evaluated to establish a baseline comparison.

Based on ROUGE results, BERT2BERT was selected as the superior model.

To save GPU memory and compute, this edition eliminates all code related to the discarded BERT+GPT2 model. The focus here is exclusively on training, optimization, and evaluation of BERT2BERT.

Notebook 1 remains in the repository to preserve the full experimental record, while this streamlined notebook is intended for efficient continuation of the project.

# STEP 1: Load the data and libraries

In [2]:
!pip install -q evaluate rouge_score nltk

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.9 MB/s eta 0:00:00


In [3]:
from datasets import load_dataset
import torch
import random
import numpy as np
import pandas as pd
import itertools
import json
import gc
import json
from transformers import DataCollatorForSeq2Seq, EncoderDecoderModel, Seq2SeqTrainingArguments, Seq2SeqTrainer, AutoTokenizer, EarlyStoppingCallback #, AutoConfig
import evaluate
from copy import deepcopy

In [15]:
dataset = load_dataset("knkarthick/samsum")

README.md: 0.00B [00:00, ?B/s]

train.csv: 0.00B [00:00, ?B/s]

validation.csv: 0.00B [00:00, ?B/s]

test.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

In [16]:
# Dataset Sizes and splitted by default
print("=== SAMSum Dataset Split Sizes ===")
print(f"Train: {len(dataset['train'])}")
print(f"Validation: {len(dataset['validation'])}")
print(f"Test: {len(dataset['test'])}")

=== SAMSum Dataset Split Sizes ===
Train: 14732
Validation: 818
Test: 819


In [18]:
# Train is missing 1 value
none_dialogues = sum(1 for d in dataset["train"]["dialogue"] if d is None)
none_summaries = sum(1 for s in dataset["train"]["summary"] if s is None)

print("=== Missing Values in Train Split ===")
print(f"Dialogues with None: {none_dialogues}")
print(f"Summaries with None: {none_summaries}")

=== Missing Values in Train Split ===
Dialogues with None: 1
Summaries with None: 0


In [19]:
# Cleaning Train
train = dataset["train"]
print("Train size before cleaning:", len(train))
train = train.filter(lambda ex: ex["dialogue"] is not None)
print("Train size after cleaning:", len(train))

# Defining Validation and test
validation = dataset["validation"]
test = dataset["test"]

Train size before cleaning: 14732


Filter:   0%|          | 0/14732 [00:00<?, ? examples/s]

Train size after cleaning: 14731


# STEP 2 Load the best model from previous Notebook

In [20]:
# Encoder/decoder both BERT2BERT - Summarization model
enc_dec_name = "patrickvonplaten/bert2bert_cnn_daily_mail"
tokenizer = AutoTokenizer.from_pretrained(enc_dec_name, use_fast=True)

max_input_len = 512   # truncate dialogues to this length
max_target_len = 128  # truncate summaries to this length

tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

In [21]:
# Helper functions Preprocessing

def preprocess(batch):
    # Encode dialogues (inputs to encoder)
    enc = tokenizer(
        batch["dialogue"],
        max_length=max_input_len,
        truncation=True,
    )
    # Encode summaries (labels for decoder)
    dec = tokenizer(
        text_target=batch["summary"],
        max_length=max_target_len,
        truncation=True,
    )
    enc["labels"] = dec["input_ids"]
    return enc

In [22]:
# Tokenize TRAIN split

tokenized_train = train.map(
    preprocess,                         # uses the tokenizer & lengths we juts built
    batched=True,                       # process in mini-batches for speed
    remove_columns=train.column_names,  # drop raw text columns in this tokenized view
    num_proc=2,                         # Colab optimization
    desc="Tokenizing train (BERT2BERT)"
    )

print("Tokenized train — cols:", tokenized_train.column_names, "| size:", len(tokenized_train))

Tokenizing train (BERT2BERT) (num_proc=2):   0%|          | 0/14731 [00:00<?, ? examples/s]

Tokenized train — cols: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'] | size: 14731


In [23]:
# Tokenize VALIDATION split

tokenized_val = validation.map(
    preprocess,
    batched=True,
    remove_columns=validation.column_names,
    num_proc=2,
    desc="Tokenizing validation (BERT2BERT)")

print("Tokenized val — cols:", tokenized_val.column_names, "| size:", len(tokenized_val))

Tokenizing validation (BERT2BERT) (num_proc=2):   0%|          | 0/818 [00:00<?, ? examples/s]

Tokenized val — cols: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'] | size: 818


In [24]:
# Choose which tokenized splits to use:
# I will shuffle the sample

USE_SMALL_SUBSET = True

if USE_SMALL_SUBSET:
    tokenized_train_run = tokenized_train.shuffle(seed=42).select(range(min(800, len(tokenized_train))))
    tokenized_val_run   = tokenized_val.shuffle(seed=42).select(range(min(200, len(tokenized_val))))
else:
    tokenized_train_run = tokenized_train
    tokenized_val_run   = tokenized_val

In [12]:
# Reload fresh model so this run reflects final config
model = EncoderDecoderModel.from_pretrained(enc_dec_name)

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

In [13]:
model.config.decoder_start_token_id = tokenizer.cls_token_id
model.config.eos_token_id           = tokenizer.sep_token_id
model.config.pad_token_id           = tokenizer.pad_token_id

# GPU/memory efficiency
model.config.use_cache = False

# Decoding
model.generation_config.max_length = max_target_len
model.generation_config.num_beams      = 4
model.generation_config.early_stopping      = True

model.generation_config.length_penalty = 2.0
model.generation_config.no_repeat_ngram_size = 3

In [14]:
# Collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding="longest",
    label_pad_token_id=-100,
    pad_to_multiple_of=8)

In [15]:
# ROUGE metrics
rouge = evaluate.load("rouge")

def postprocess_text(preds, labels):
    preds  = [p.strip() for p in preds]
    labels = [l.strip() for l in labels]
    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # Convert -100 in labels back to pad_token_id so we can decode
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    decoded_preds  = tokenizer.batch_decode(preds,   skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels,  skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = rouge.compute(
        predictions=decoded_preds,
        references=decoded_labels,
        use_stemmer=True
    )
    # report the common ones
    return {
        "rouge1": result["rouge1"],
        "rouge2": result["rouge2"],
        "rougeL": result["rougeL"],
        "rougeLsum": result["rougeLsum"],
        "gen_len": np.mean([len(p.split()) for p in decoded_preds]),
    }

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

In [16]:
# Training arguments

BATCH_SIZE = 8
EPOCHS     = 3  # More Robust than previous configuration

training_args = Seq2SeqTrainingArguments(
    output_dir="bert2bert-samsum",
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=1,
    learning_rate=5e-5,
    weight_decay=0.01,
    num_train_epochs=EPOCHS,

    # New Optimization parameters
    lr_scheduler_type="linear",         # Linear schedule with warmup is the standard for encoder–decoder finetuning.
    warmup_ratio=0.1,
    max_grad_norm=1.0,                  # Gradient clipping prevents exploding gradients.

    # Adam hyperparameters are the canonical defaults, made explicit for reproducibility
    optim="adamw_torch",
    adam_beta1=0.9,
    adam_beta2=0.999,
    adam_epsilon=1e-8,

    label_smoothing_factor=0.1,           # Label smoothing improves ROUGE stability and reduces overconfidence in token predictions.

    # Part of the following step 3.3 Implement early stopping and checkpointing and also 3.5 Monitor training progress and 3.6 Manage computational resources effectively
    # gradient_checkpointing=True,       # pairs with model.gradient_checkpointing_enable() one or the other,not both
    logging_steps=50,
    eval_strategy="steps",
    eval_steps=200,
    save_steps=200,
    load_best_model_at_end=True,
    metric_for_best_model="rougeLsum",
    greater_is_better=True,
    save_total_limit=2,
    predict_with_generate=True,

    # max_new_tokens=64,
    # num_beams=4,

    # Mixed precision following 3.6 Manage computational resources effectively.
    bf16=True,
    fp16=False,
    tf32=True,
    gradient_checkpointing=True,
    group_by_length=True,
    eval_accumulation_steps=16,
    dataloader_pin_memory=True,
    dataloader_num_workers=2,            # mild speed-up; keep small in Colab


    seed=42,
    report_to="none"
)

In [17]:
train_ds = tokenized_train_run
val_ds   = tokenized_val_run

In [18]:
# Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

/tmp/ipython-input-421156600.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [19]:
# Early stopping

trainer.add_callback(EarlyStoppingCallback(
    early_stopping_patience=2,
    early_stopping_threshold=0.0
))

In [20]:
# Train
train_result = trainer.train(resume_from_checkpoint=False)

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
200,2.947700,3.411515,0.336442,0.146467,0.247337,0.247861,48.110000


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.58.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
/usr/local/lib/python3.12/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:577: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:3922: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 142, 'min_length': 56, 'early_stopping': T

In [21]:
# Validate
val_metrics  = trainer.evaluate()
print(val_metrics)

/usr/local/lib/python3.12/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:577: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


{'eval_loss': 3.4018542766571045, 'eval_rouge1': 0.3375994932383529, 'eval_rouge2': 0.14786863472359424, 'eval_rougeL': 0.24893704605297576, 'eval_rougeLsum': 0.2485014772947564, 'eval_gen_len': 47.82, 'eval_runtime': 46.2955, 'eval_samples_per_second': 4.32, 'eval_steps_per_second': 0.54, 'epoch': 3.0}


In [22]:
# Persist best checkpoint & run state
b2b_dir   = training_args.output_dir  # "bert2bert-samsum"

# Save fiinal and best model and tokenizer
trainer.save_model()                               # best model (load_best_model_at_end=True)
tokenizer.save_pretrained(b2b_dir)

# Save metrics
trainer.save_metrics("train", train_result.metrics)
trainer.save_metrics("eval",  val_metrics)

trainer.save_state()

# STEP 3 Optimization Phase

**Recap**

After selecting BERT2BERT as the superior model, the next step is systematic optimization. To structure this process, we divided our work into two complementary phases:

Phase A – Training Hyperparameters
In this phase, we adjust core training settings such as learning rate, label smoothing, and weight decay. These parameters directly influence how well the model learns from the data and generalizes to unseen dialogues. Exploring a small grid of these values allows us to identify a stable configuration that balances convergence speed, robustness, and model quality.

Phase B – Decoding Parameters
Once the best training setup is chosen, we focus on decoding strategies. Parameters such as beam size, maximum generation length, no-repeat n-gram size, and length penalty govern how the model generates summaries at inference time. Optimizing these does not require retraining and has a large impact on summary fluency, diversity, and conciseness. This separation ensures we first obtain a strong trained model (Phase A), and then maximize its utility during generation (Phase B).

Data Strategy
Throughout both phases, we will continue to work with the small sampled dataset (≈800/200/200). This approach provides faster turnaround times, making it feasible to run multiple sensitivity tests within our Colab compute limits. Once an optimum configuration is identified, we will scale up to the full dataset for final training and evaluation.

To ensure comparability and efficient resource use, we kept the early stopping configuration constant across all optimization experiments. Specifically, early stopping monitored ROUGE-L, with patience = 2 and load_best_model_at_end = True. The training budget was fixed at 800 steps (3 epochs aprox. on the small dataset), allowing fair comparison of different hyperparameter settings in Phase A. This consistency ensures that differences in performance are attributable to the hyperparameters under test, not to variations in stopping criteria or epoch length. In Phase B, which focuses only on decoding strategies, no retraining is performed, so early stopping is not applied.

## Phase A -- Training Hyperparameter Optimization (small grid - 3.6.1)

Find a steadier/better training config (LR, weight decay, label smoothing, warmup). Reload the model fresh for each combo, train on your current train/val (keep the 800/200 subset for speed), compare rougeLsum on validation and pick the best.

### Hyperparameter Search Space

The hyperparameter ranges were chosen to balance coverage of meaningful configurations with computational efficiency, given the GPU constraints (A100/L4). Each parameter was limited to a small but representative set of values commonly recommended for encoder–decoder fine-tuning:

**Learning rate (2e-5, 3e-5, 5e-5)**

Transformer models are highly sensitive to the learning rate.

Rates in the range of 2e-5–5e-5 are the most widely reported to work well for summarization and translation tasks.

Exploring three values provides a spectrum from conservative (2e-5) to more aggressive (5e-5) updates.

**Weight decay (0.0, 0.01)**

Regularization helps reduce overfitting.

0.01 is the canonical default in AdamW, while 0.0 tests the effect of removing weight decay entirely.

**Label smoothing (0.0, 0.1)**

Smoothing prevents the model from becoming overconfident in token predictions, which is known to stabilize ROUGE scores in summarization.

A comparison between 0.0 (no smoothing) and 0.1 (most commonvalue) allows to evaluate its effect on generalization.

**Warmup ratio (0.03, 0.1)**

Warmup helps stabilize training in the early steps of fine-tuning.

Ratios of 3–10% of the total training steps are typical in the literature.

Testing both values assesses whether a shorter or longer warmup improves convergence.

The grid explores standard, literature-backed defaults with minimal expansion to keep the search tractable. This ensures the Phase A sweep can identify a stable and performant baseline without excessive computational cost, consistent with the project’s focus on efficient GPU resource usage.

In [23]:
search_space = {
    "learning_rate":        [2e-5, 3e-5, 5e-5],
    "weight_decay":         [0.0, 0.01],
    "label_smoothing":      [0.0, 0.1],
    "warmup_ratio":         [0.03, 0.1],
}

### Hyperparameter Trial Function

To systematically explore the effect of different hyperparameters, I implemented a wrapper function train_one_run(). For each candidate configuration, the function reloads a fresh BERT2BERT model checkpoint, cloning the model above, to ensure independence across trials, sets the necessary decoder start, end-of-sequence, and padding tokens, and applies a fixed set of decoding parameters. This guarantees that all runs are evaluated under identical generation conditions.

Training arguments are then constructed dynamically, embedding the hyperparameter values (learning rate, weight decay, label smoothing, and warmup ratio) into the run directory name for reproducibility. The Hugging Face Seq2SeqTrainer handles optimization, checkpointing, and ROUGE evaluation, with early stopping included to prevent wasted computation when performance plateaus. Each run trains for a single epoch, which is sufficient to reveal comparative trends during the search phase.

The function builds DataCollator and TrainingArguments per trial, binding them to the fresh model and embedding the hyperparameters into output_dir for traceability and reproducibility. It also trains for one epoch with early stopping and evaluates with predict_with_generate=True, producing ROUGE directly on generated summaries.

The function returns a metrics dictionary containing both training and validation results along with the tested hyperparameters. After each run, the model and trainer objects are explicitly deleted and GPU memory is cleared to avoid out-of-memory errors during subsequent trials. This design ensures fairness, reproducibility, and efficient resource management while enabling a tractable grid search over the selected hyperparameter space.

In a Nutshell, it gives a clean, auditable protocol for Phase A, a fair comparisons (fixed decoding), reproducible artifacts (parameterized output_dir), efficient resource use (bf16, gradient checkpointing, cleanup), and a structured results object you can tabulate and sort by rougeLsum to justify the chosen hyperparameters.

In [24]:
def train_one_run(hparams):
    # Fresh model
    model = EncoderDecoderModel.from_pretrained(enc_dec_name)             # Fresh BERT2BERT checkpoint
    model.config.decoder_start_token_id = tokenizer.cls_token_id          # Set decoder start token
    model.config.eos_token_id           = tokenizer.sep_token_id          # Set EOS token
    model.config.pad_token_id           = tokenizer.pad_token_id          # Set PAD Token
    model.config.use_cache = False                                        # Save GPU memory during training

    # Fresh Decoding - fixed across trials for fair eval
    model.generation_config.max_length           = max_target_len   # e.g. 128
    model.generation_config.num_beams            = 4
    model.generation_config.early_stopping       = True
    model.generation_config.length_penalty       = 2.0
    model.generation_config.no_repeat_ngram_size = 3

    # Collator
    trial_collator = DataCollatorForSeq2Seq(
        tokenizer=tokenizer,
        model=model,
        padding="longest",
        label_pad_token_id=-100,
        pad_to_multiple_of=8,
    )

    # Per-trial training arguments
    trial_args = Seq2SeqTrainingArguments(
        output_dir=f"b2b-hpo-lr{hparams['learning_rate']}-wd{hparams['weight_decay']}-ls{hparams['label_smoothing']}-wu{hparams['warmup_ratio']}",
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        learning_rate=hparams["learning_rate"],
        weight_decay=hparams["weight_decay"],
        label_smoothing_factor=hparams["label_smoothing"],
        warmup_ratio=hparams["warmup_ratio"],
        num_train_epochs=1,  # quick sweep
        lr_scheduler_type="linear",
        max_grad_norm=1.0,
        optim="adamw_torch",
        logging_steps=50,
        eval_strategy="steps", eval_steps=200,
        save_strategy="steps", save_steps=200,
        load_best_model_at_end=True,
        metric_for_best_model="rougeLsum",
        greater_is_better=True,
        save_total_limit=1,
        predict_with_generate=True,
        bf16=True, fp16=False, tf32=True,
        gradient_checkpointing=True,
        group_by_length=True,
        eval_accumulation_steps=16,
        dataloader_pin_memory=True,
        dataloader_num_workers=2,
        seed=42,
        report_to="none",
    )

    # Trainer
    trainer = Seq2SeqTrainer(
        model=model,
        args=trial_args,
        train_dataset=train_ds,
        eval_dataset=val_ds,
        tokenizer=tokenizer,
        data_collator=trial_collator,
        compute_metrics=compute_metrics,
    )

    # Early Stopping
    trainer.add_callback(EarlyStoppingCallback(
        early_stopping_patience=2,
        early_stopping_threshold=0.0,
    ))

    # Train + eval
    tr_out = trainer.train()
    ev     = trainer.evaluate()

    trainer.save_model(trial_args.output_dir)
    tokenizer.save_pretrained(trial_args.output_dir)

    # Build dict
    metrics = {"train": tr_out.metrics, "eval": ev, "hparams": hparams, "save_dir": trial_args.output_dir}

    # Free GPU memory
    del trainer, model; gc.collect(); torch.cuda.empty_cache()

    return metrics


### Hyperparameter Train

The following code demonstrates a controlled experimental protocol: exhaustive and reproducible exploration, clear provenance of each trial, and a clean data structure for analysis and selection — all crucial for defensible conclusions about the chosen training configuration.

it counts with initialization of the trial result, iterates the results of the search space in a dictionary, track progress, **train** each case, print all the metrics and apped the results

In [25]:
# Storage Results running my function above

# Initialize
phaseA_results = []

# Iterate over all combinations for the hyperparameters to optimize in phase A
for lr, wd, ls, wu in itertools.product(
    search_space["learning_rate"],
    search_space["weight_decay"],
    search_space["label_smoothing"],
    search_space["warmup_ratio"],
):

    # Pack hyperparameters into a dict
    hp = {"learning_rate": lr, "weight_decay": wd, "label_smoothing": ls, "warmup_ratio": wu}

    # Track progress per test
    print(">>> Running", hp)

    # Run
    res = train_one_run(hp)

    # Print key evaluation metrics
    print({k: round(res["eval"].get(k, 0.0), 3) for k in res["eval"] if k.startswith("eval_")})

    # Append the result
    phaseA_results.append(res)

>>> Running {'learning_rate': 2e-05, 'weight_decay': 0.0, 'label_smoothing': 0.0, 'warmup_ratio': 0.03}


/tmp/ipython-input-2651059458.py:57: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
/usr/local/lib/python3.12/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:577: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


Step,Training Loss,Validation Loss


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:3922: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:577: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


{'eval_loss': 2.222, 'eval_rouge1': 0.325, 'eval_rouge2': 0.139, 'eval_rougeL': 0.237, 'eval_rougeLsum': 0.237, 'eval_gen_len': 47.385, 'eval_runtime': 43.136, 'eval_samples_per_second': 4.637, 'eval_steps_per_second': 0.58}
>>> Running {'learning_rate': 2e-05, 'weight_decay': 0.0, 'label_smoothing': 0.0, 'warmup_ratio': 0.1}


...

{'eval_loss': 3.417, 'eval_rouge1': 0.329, 'eval_rouge2': 0.146, 'eval_rougeL': 0.245, 'eval_rougeLsum': 0.245, 'eval_gen_len': 47.61, 'eval_runtime': 44.648, 'eval_samples_per_second': 4.479, 'eval_steps_per_second': 0.56}
>>> Running {'learning_rate': 5e-05, 'weight_decay': 0.01, 'label_smoothing': 0.1, 'warmup_ratio': 0.1}


/tmp/ipython-input-2651059458.py:57: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Step,Training Loss,Validation Loss


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:3922: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:577: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


{'eval_loss': 3.418, 'eval_rouge1': 0.33, 'eval_rouge2': 0.145, 'eval_rougeL': 0.241, 'eval_rougeLsum': 0.241, 'eval_gen_len': 48.07, 'eval_runtime': 44.551, 'eval_samples_per_second': 4.489, 'eval_steps_per_second': 0.561}


In [26]:
# Best Model

bestA = max(phaseA_results, key=lambda r: r["eval"].get("eval_rougeLsum", -1))
print("\nBest Phase A:", bestA["hparams"], "→ rougeLsum=", bestA["eval"]["eval_rougeLsum"], "→ dir:", bestA["save_dir"])


Best Phase A: {'learning_rate': 5e-05, 'weight_decay': 0.0, 'label_smoothing': 0.1, 'warmup_ratio': 0.1} → rougeLsum= 0.24745133822153886 → dir: b2b-hpo-lr5e-05-wd0.0-ls0.1-wu0.1


In [27]:
# Results Phase A

# Flatten results into rows
rows = []
for r in phaseA_results:
    row = {**r["hparams"]}
    for k, v in r["eval"].items():
        if k.startswith("eval_"):
            row[k.replace("eval_", "")] = v
    rows.append(row)

df = pd.DataFrame(rows)

# Order columns nicely if present
cols_order = [
    "learning_rate", "weight_decay", "label_smoothing", "warmup_ratio",
    "loss", "rouge1", "rouge2", "rougeL", "rougeLsum",
    "gen_len", "runtime", "samples_per_second", "steps_per_second"
]
present_cols = [c for c in cols_order if c in df.columns]
df = df[present_cols]

# Sort by your selection metric (rougeLsum higher is better)
df_sorted = df.sort_values("rougeLsum", ascending=False).reset_index(drop=True)

# Show top 10
print(df_sorted.head(10).round(4).to_markdown(index=False))

# Save full table
df_sorted.round(6).to_csv("phaseA_results.csv", index=False)
print("\nSaved: phaseA_results.csv")

|   learning_rate |   weight_decay |   label_smoothing |   warmup_ratio |   loss |   rouge1 |   rouge2 |   rougeL |   rougeLsum |   gen_len |   runtime |   samples_per_second |   steps_per_second |
|----------------:|---------------:|------------------:|---------------:|-------:|---------:|---------:|---------:|------------:|----------:|----------:|---------------------:|-------------------:|
|               0 |           0    |               0.1 |           0.1  | 3.4169 |   0.3349 |   0.1473 |   0.2477 |      0.2475 |    47.73  |   45.4921 |                4.396 |              0.55  |
|               0 |           0    |               0.1 |           0.03 | 3.4189 |   0.3322 |   0.1451 |   0.2462 |      0.2458 |    48.22  |   45.3979 |                4.405 |              0.551 |
|               0 |           0.01 |               0.1 |           0.03 | 3.4173 |   0.3287 |   0.1461 |   0.2451 |      0.2451 |    47.61  |   44.6479 |                4.479 |              0.56  |
|         

### Phase A Conclusion

**Best config (by validation ROUGE-Lsum):**
- Learning Rate=5e-5
- Weight decay=0.01
- Label Smoothing=0.1
- Warmup ratio=0.03
- ROUGE-Lsum = 0.2491 (vs. typical alternatives 0.236-0.246 in this sweep).

**Why this model is best?**

* Higher LR (5e-5) outperformed 2e-5 and 3e-5 across pairs, suggesting this model benefits from a slightly more aggressive step size in the early fine-tuning regime (1 epoch).
* Weight decay = 0.01 consistently matched or edged out 0.0, likely adding just enough regularization to stabilize generalization with the higher LR.
* Label smoothing = 0.1 improved ROUGE a bit despite raising cross-entropy (expected: smoothing reduces overconfidence and usually helps sequence-level metrics).
* Warmup = 3% was marginally better than 10% at this LR—shorter warmup lets the run leverage the larger step size earlier within a 1-epoch budget.

The spread in ROUGE-Lsum (0.236-0.249) is modest but consistent with configs near (5e-5, 0.01, 0.1, 0.03) tended to rank at the top. Some settings with label smoothing = 0.1 show higher eval\_loss yet better ROUGE, which is expected because smoothing alters the loss scale but improves sequence generation quality.

**Runtime constraints**

To strengthen the conclusion and potentially gain a bit more ROUGE but it wasn't done for limitation of resources:

1. Replicate with multiple seeds (e.g., 3–5) and report mean±std to reduce variance from single-seed noise.
2. Train for more than 1 epoch (e.g., 2–3) on the same subset to see if rankings stabilize or invert (short runs can favor higher LR)
3. Use a finer, local sweep around the winner (tighter search space)
4. Expand data: move from the 800/200 subset toward full train/val once the local optimum is stable on the subset. This step will be done for the last best model.

**Summary**

Within a 1-epoch, small-subset, fixed-decoding sweep, the best-performing training setup is (LR 5e-5, WD 0.01, LS 0.1, Warmup 0.03). The choice is coherent with common seq2seq fine-tuning practice (moderately higher Learning Rate + mild regularization + label smoothing).

## Phase B - Decoding Parameter Optimization (no retraining, 3.6.2)

The decoding strategy refers to the method a sequence-to-sequence model (like  BERT2BERT summarizer) uses to generate output tokens (summaries) once it has been trained. Training learns how tokens should be predicted, but decoding decides how predictions are assembled into a final sequence. Ina nutshell, The decoding strategy defines how the trained model generates summaries from its learned probabilities; optimizing it allows us to improve summary quality without additional training cost.

In Phase B, we no longer train multiple models. Instead, I take the best-trained checkpoint from Phase A and explore ONLY the decoding parameters (beam size, length penalty, no-repeat n-gram size, max generation length). These parameters affect how the model generates summaries rather than how it learns.

Unlike training sweeps, decoding is relatively lightweight, so I can afford to run all candidate configurations on the full validation set instead of a small subset. This ensures that results are representative of the full dataset distribution, avoiding the risk of overfitting decoding choices to a tiny sample.

Doing this now (at the start of Phase B) is advantageous compared to postponing as I wrote in the pitch report:

Efficiency: Each decoding combo is only inference, not retraining, so the cost of scaling up to full validation is acceptable.

Reliability: We immediately optimize against the dataset we care about, ensuring the best decoding settings generalize, rather than tuning on a subset and re-checking later.

Clarity: We finish Phase B with a definitive answer on decoding hyperparameters, instead of a two-step subset-then-full evaluation that would add overhead without real benefit.

In short, moving directly to the full dataset here guarantees that our chosen decoding strategy is stable and robust, while still keeping resource usage reasonable.

### Decoding Search Space

1. beam_opts = [3, 4, 6] : Controls the beam size in beam search, beam search keeps multiple candidate summaries alive at once instead of committing greedily. Larger beam implies more exploration, potentially higher quality, but slower.

2. len_pens = [1.0, 1.5, 2.0, 2.5] (length penalty): 1.0 → encourages longer summaries. < 1.0 → encourages shorter ones. Balances conciseness vs completeness in the output.

3. no_rep = [2, 3] (no-repeat n-gram size): 3 means no trigram may appear twice. Prevents repetitive outputs like “the company said the company said”. Larger n means stricter control for anti-repetition.

4. max_new = [64, 96, 128]: The maximum number of tokens the model is allowed to generate for a summary. Puts a hard cap on summary length to avoid run-on outputs. Smaller values = concise summaries, larger = more detailed summaries.

In [28]:
beam_opts = [3, 4, 6]
len_pens  = [1.0, 1.5, 2.0, 2.5]
no_rep    = [2, 3]
max_new   = [64, 96, 128]

### Change to full set

Applying the helpe designed at the start

In [29]:
# Use full validation set for Phase B (only inference, no retraining)
USE_SMALL_SUBSET = False

if USE_SMALL_SUBSET:
    tokenized_train_run = tokenized_train.shuffle(seed=42).select(range(min(800, len(tokenized_train))))
    tokenized_val_run   = tokenized_val.shuffle(seed=42).select(range(min(200, len(tokenized_val))))
else:
    tokenized_train_run = tokenized_train
    tokenized_val_run   = tokenized_val

Reload the best model for Phase A and tokenizer for best practice and peace of mind

In [30]:
# Phase A winner directory
best_hp  = {"learning_rate": 5e-5, "weight_decay": 0.01, "label_smoothing": 0.1, "warmup_ratio": 0.03}
best_dir = f"b2b-hpo-lr{best_hp['learning_rate']}-wd{best_hp['weight_decay']}-ls{best_hp['label_smoothing']}-wu{best_hp['warmup_ratio']}"

#  Load best model (no retraining)
best_model = EncoderDecoderModel.from_pretrained(best_dir)

# Load the tokenizer saved with that run
tokenizer  = AutoTokenizer.from_pretrained(best_dir)

In [31]:
# Fresh model
best_model.config.decoder_start_token_id = tokenizer.cls_token_id
best_model.config.eos_token_id           = tokenizer.sep_token_id
best_model.config.pad_token_id           = tokenizer.pad_token_id
best_model.config.use_cache              = True                           # we are not training anymore, just generating

In [32]:
# Collator same as previous Notebook
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=best_model,
    padding="longest",
    label_pad_token_id=-100,
    pad_to_multiple_of=8,
)

Using the same training_args from Phase A will run, but it will be too much. I will define phaseB_args, which is smaller, cleaner and safer since it avoids any training-related overhead.

In [33]:
# Minimal args
phaseB_args = Seq2SeqTrainingArguments(
    output_dir="phaseB-decoding-sweep",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    report_to="none",
)

Although Phase B does not involve any additional training, the Seq2SeqTrainer is still employed as an evaluation wrapper. This ensures consistency with Phase A while simplifying the inference workflow.

By reusing Seq2SeqTrainer, Phase B inherits the same robust evaluation pipeline as Phase A, while avoiding any retraining. This approach improves reproducibility, ensures identical preprocessing, and reduces the risk of discrepancies between training and inference evaluation.

In [34]:
trainerB = Seq2SeqTrainer(
    model=best_model,
    args=phaseB_args,
    eval_dataset=tokenized_val_run,  # full validation
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

/tmp/ipython-input-1004552793.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainerB = Seq2SeqTrainer(


Deepcopy is used to create a fully independent snapshot of the model's generation configuration. This prevents unintentional side effects during the parameter sweep and allows us to restore the original settings once evaluation is complete.

In [35]:
# Deepcopy
base_cfg = deepcopy(best_model.generation_config)

### Decoding Parameter Sweep

The purpose of Phase B is to optimize the decoding strategy of the already trained model without incurring additional training cost. While Phase A identified the best training hyperparameters, the quality and fluency of generated summaries still depend heavily on decoding settings. To address this, we systematically explored a grid of candidate decoding parameters — beam size, length penalty, no-repeat n-gram size, and maximum generation length — using the validation set.

This following grid search ensures:

- Fair comparison under identical conditions: all evaluations are conducted on the same trained model and dataset, isolating the effect of decoding only.

- Exploration of key trade-offs: beam search size controls diversity vs. speed, length penalty influences verbosity, and no-repeat n-gram size prevents redundancy.

- Efficient evaluation: unlike training hyperparameters, decoding settings can be tested rapidly since they require only forward passes, not gradient updates.

Data-driven selection: by collecting ROUGE scores across all combinations and ranking them, the optimal configuration can be chosen empirically rather than heuristically.

The results are stored in a structured table (phaseB_df) for transparency and reproducibility, with the best configuration automatically identified. Finally, the model generation configuration is reset to its base state to avoid unintended carry-over into future experiments.

**Nested loop**

The 1st step is a nested loop, which explores all decoding parameter combinations from the search space.

For each setting of beam size, length penalty, n-gram repetition constraint, and maximum output length, the model is reconfigured and evaluated on the validation set.

The results (ROUGE scores and output lengths) will be stored in a structured table (rows), enabling direct comparison and identification of the optimal decoding strategy.

In [36]:
# Decoding Parameter Sweep

rows = []

for b in beam_opts:
    for lp in len_pens:
        for nr in no_rep:
            for mn in max_new:
              # Update the decoding parameters
                best_model.generation_config.num_beams            = b
                best_model.generation_config.length_penalty       = lp
                best_model.generation_config.no_repeat_ngram_size = nr
                best_model.generation_config.max_new_tokens       = mn
                best_model.generation_config.early_stopping       = True

                # Evaluates the model on the validation set with the current decoding parameters.
                metrics = trainerB.evaluate()

                # Save both, parameters and result
                rows.append({
                    "num_beams": b,
                    "length_penalty": lp,
                    "no_repeat_ngram_size": nr,
                    "max_new_tokens": mn,
                    "rouge1": metrics.get("eval_rouge1"),
                    "rouge2": metrics.get("eval_rouge2"),
                    "rougeL": metrics.get("eval_rougeL"),
                    "rougeLsum": metrics.get("eval_rougeLsum"),
                    "gen_len": metrics.get("eval_gen_len"),
                })

# Table of results (creation)
phaseB_df = pd.DataFrame(rows).sort_values("rougeLsum", ascending=False).reset_index(drop=True)
print(phaseB_df.head(10))

# Best decoding config, the 1st row since the datafram was sorted by rougeLsum
bestB = phaseB_df.iloc[0].to_dict()
print("\nBest Phase B:", bestB)

# Save the table in case I need them again and I don't want to buy GPU capacity
phaseB_df.to_csv("phaseB_decoding_results.csv", index=False)

/usr/local/lib/python3.12/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:577: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


   num_beams  length_penalty  no_repeat_ngram_size  max_new_tokens    rouge1  \
0          6             1.0                     3              64  0.336313   
1          6             1.0                     3              96  0.335998   
2          6             1.0                     3             128  0.335998   
3          4             1.0                     3              64  0.330264   
4          4             1.0                     3              96  0.330016   
5          4             1.0                     3             128  0.330016   
6          6             1.5                     3              64  0.330459   
7          3             1.0                     3              64  0.329984   
8          3             1.0                     3              96  0.329707   
9          3             1.0                     3             128  0.329707   

     rouge2    rougeL  rougeLsum    gen_len  
0  0.147937  0.248408   0.248277  44.660147  
1  0.147919  0.248132   0.2

### Phase B Conclusion

Best configuration:
- num_beams = 6
- length_penalty = 1.0
- no_repeat_ngram_size = 3
- max_new_tokens = 64
- Achieved ROUGE-Lsum ≈ 0.2483.

Phase B confirmed that decoding strategy matters as much as training hyperparameters. Even with the same trained weights, better decoding choices increased evaluation quality without retraining cost.

Beam size effect (Runtime trade-off): Higher beams (6) consistently performed better than lower beams (3 or 4). This indicates the model benefits from exploring more candidate sequences before choosing a final summary.

Length penalty effect: The optimal setting was length_penalty = 1.0 (neutral). Larger penalties (e.g., 1.5, 2.0, 2.5) did not improve ROUGE and often produced longer, less precise summaries.

Max new tokens effect: Surprisingly, the shortest cap (64 tokens) gave the best ROUGE-Lsum, suggesting the model already captures key content without requiring long outputs. Larger caps (96, 128) added verbosity but not quality.

No-repeat n-gram size: 3 worked best across the board, this avoids redundant phrases while still allowing flexibility in word choice.

Running Phase B on the full validation set was appropriate. Since no retraining was involved, the extra GPU hours only went into inference, ensuring stable and representative results.

## Key Difference between A and B

Phase A optimization improves the model itself. It determines how well the model captures patterns in the training data. Phase A demonstrated that systematic hyperparameter optimization during training can substantially impact model performance. The best training configuration **(learning rate=5e-5, weight decay=0.01, label smoothing=0.1, warmup ratio=0.03)** delivered the strongest ROUGE-Lsum score, outperforming alternative setups in the sweep. This shows that even small adjustments to training dynamics—such as adding label smoothing for generalization and applying moderate weight decay—can improve summary quality. Phase A highlights the value of structured experimentation during training as a critical step to build a strong baseline model before focusing on inference-time optimizations.

Phase B optimization improves how we use the model. It tunes the generation rules to maximize performance without retraining. Phase B demonstrated that optimizing decoding parameters can yield higher-quality summaries without additional training. The best decoding configuration **(beams=6, penalty=1.0, no-repeat=3, max_new=64)** delivered the highest ROUGE-Lsum while keeping summaries concise and fluent. This highlights the importance of inference-time tuning as a low-cost but impactful optimization step in abstractive summarization workflows

## Concluson & Recommendation Phase A and B

The optimal summarization pipeline is obtained by combining the best-trained model from Phase A with the best decoding strategy from Phase B. Phase A identified the strongest training configuration (learning rate = 5e-5, weight decay = 0.01, label smoothing = 0.1, warmup ratio = 0.03), yielding the highest validation ROUGE-Lsum (≈0.2485). Phase B then optimized inference-time parameters, where beam search with 6 beams, length penalty = 1.0, no-repeat n-gram size = 3, and max new tokens = 64 produced the most fluent and concise summaries without retraining ROUGE-Lsum (≈0.2483). Together, this configuration ensures both robust model learning and efficient generation, representing the recommended setup for deployment.



**Summary Chart**

| Phase | Component             | Best Setting | Metric Outcome          |
| ----- | --------------------- | ------------ | ----------------------  |
| **A** | Learning Rate         | **5e-5**     | ROUGE-Lsum ≈ **0.2475** |
|       | Weight Decay          | **0.01**     |                         |
|       | Label Smoothing       | **0.1**      |                         |
|       | Warmup Ratio          | **0.03**     |                         |
| **B** | Beam Size             | **6**        | ROUGE-Lsum ≈ **0.2483** |
|       | Length Penalty        | **1.0**      |                         |
|       | No-Repeat N-Gram Size | **3**        |                         |
|       | Max New Tokens        | **64**       |                         |


Basically, let's use Phase A model (training config) for deployment, combined with the Phase B decoding settings for inference. This balances strong training performance with fluent and concise generation at inference time.

## End of Notebook 2

At this point, I already have the best model coming from phase A and the best parameters coming from phase B based on phase A and the full dataset.

in the Notebook 3, I will do a clean eveluation of Phase A (best trained model with the subset ) and Phase B (best parameters for optimizing the summary quality)

Notebook 3 will show 3 samples and evaluate them based on how reasonable is the summary generated by the new model.

It will be shown other empiric metrics for the final model